<a href="https://colab.research.google.com/github/Miki-Bagna/BertSum/blob/main/testing_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai==1.0.61

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=32246f06ac6ac323c6c2460d96efad02252dd1f9888c78b349c3e27f525f25e0
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.13
    Uninstalling fastai-2.7.13:
      Successfully uninstalled fastai-2.7.13


In [2]:
!git clone https://github.com/huggingface/transformers
!cd transformers
!pip install *

Cloning into 'transformers'...
remote: Enumerating objects: 176229, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 176229 (delta 71), reused 100 (delta 50), pack-reused 176081
Receiving objects: 100% (176229/176229), 207.82 MiB | 7.60 MiB/s, done.
Resolving deltas: 100% (120939/120939), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 971.0/971.0 kB 6.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/gdrive')
import csv

Mounted at /gdrive


***Per iniziare se non riconosce "modeling_bertabs" va spostata la cartella "/content/transformers/examples/research_projects/bertabs" dentro all cartella "/usr/local/lib/python3.10/dist-packages/transformers"***

In [4]:
!pip install torch torchvision torchaudio

In [7]:
from fastai.text import *
from fastai.callbacks import *
import glob
import torch
from transformers.bertabs.modeling_bertabs import BertAbsConfig, BertAbs, build_predictor
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, SequentialSampler
from torch.nn import functional as F
from transformers import BertTokenizer, BertModel
import logging
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)
logging.getLogger().setLevel(100)

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

args = Namespace(
    adam_b1=0.9,
    adam_b2=0.999,
    alpha=0.95,
    batch_size=8,
    beam_size=5,
    block_size=512,
    block_trigram=True,
    data_path="/gdrive/MyDrive/testing/data/test.csv",
    device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    max_length=200,
    min_length=50,
    model_name="temp",
    stories_folder='/gdrive/MyDrive/testing/data/my_own_stories/',
    subset=400,
    train_pct=0.8
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class SummarisationDataset(Dataset):
    def __init__(self, path, subset=None):
        if path.endswith('.file'):
            self.dataset = pd.read_feather(path)
        if path.endswith('.csv'):
            self.dataset = pd.read_csv(path)

        if subset:
            self.dataset = self.dataset.iloc[:subset]

    def __len__(self):
         return self.dataset.shape[0]

    def __getitem__(self, idx):
        title = ""
        article = self.dataset.iloc[idx]['source']
        article = [s.strip()+'.' for s in article.split('.')]
        article = [self.add_missing_period(line) for line in article if len(line) > 0]
        article = [s for s in article if s not in ['..', '.']]

        summary = self.dataset.iloc[idx]['target']
        summary = [s.strip()+'.' for s in summary.split('.')]
        summary = [self.add_missing_period(line) for line in summary if len(line) > 0]
        summary = [s for s in summary if s not in ['..', '.']]

        return title, article, summary

    def add_missing_period(self, line):
        END_TOKENS = [".", "!", "?", "...", "'", "`", '"', u"\u2019", u"\u2019", ")"]
        if line.startswith("@highlight"):
            return line
        if line[-1] in END_TOKENS and len(line):
            return line
        return line + "."


def encode_text(text, tokenizer, symbols, is_summary=False):
    if is_summary:
        encoded = [tokenizer.encode(s)[1:-1] for s in text]
    else:
        encoded = [tokenizer.encode(s) for s in text]

    flattened = [item for sublist in encoded for item in sublist]

    if is_summary:
        return [symbols['BOS']] + flattened + [symbols['EOS']]

    return flattened

def create_seg_embs(encoded_text, tokenizer):
    segment_embeddings = []
    sentence_num = 0
    for item in encoded_text:
        segment_embeddings.append(sentence_num % 2)
        if item == tokenizer.sep_token_id:
            sentence_num += 1

    return segment_embeddings

def pad(encoded_text, seq_length, tokenizer, symbols, is_summary=False):
    if len(encoded_text) > seq_length:
        if is_summary:
            encoded_text = encoded_text[:seq_length]
        else:
            sent_sep_idxs = [idx for idx, t in enumerate(encoded_text) if t == tokenizer.sep_token_id and idx < seq_length]
            last_sent_sep_idx = min(max(sent_sep_idxs)+1 if (len(sent_sep_idxs) > 0) else seq_length, seq_length)
            encoded_text = encoded_text[:last_sent_sep_idx]

    if len(encoded_text) < seq_length:
        encoded_text.extend([tokenizer.pad_token_id] * (seq_length - len(encoded_text)))


    if is_summary:
        encoded_text += [tokenizer.pad_token_id]

    return encoded_text

def create_mask(text_tensor):
    mask = torch.zeros_like(text_tensor)
    mask[text_tensor != tokenizer.pad_token_id] = 1
    return mask

def collate_function(data, tokenizer, symbols, block_size, training):
    encoded_stories = [encode_text(story, tokenizer, symbols) for _, story, summary in data]
    encoded_summaries = [encode_text(summary, tokenizer, symbols, True) for _, story, summary in data]
    story_segembs = [create_seg_embs(s, tokenizer) for s in encoded_stories]

    padded_stories = torch.tensor([pad(s, block_size, tokenizer, symbols) for s in encoded_stories]).long()
    padded_summaries = torch.tensor([pad(s, block_size, tokenizer, symbols, True) for s in encoded_summaries]).long()
    padded_segembs = torch.tensor([pad(s, block_size, tokenizer, symbols) for s in story_segembs]).long()

    stories_mask = create_mask(padded_stories)
    summaries_mask = create_mask(padded_summaries)

    if training:
        return [padded_stories, padded_summaries, padded_segembs, stories_mask, summaries_mask], padded_summaries[:,1:]
    else:
        Batch = namedtuple("Batch", ["document_names", "batch_size", "src", "segs", "mask_src", "tgt_str"])
        names = [name for name, _, _ in data]
        summaries = [" ".join(summary_list) for _, _, summary_list in data]
        batch = Batch(
            document_names=names,
            batch_size=len(encoded_stories),
            src=padded_stories.to(args.device),
            segs=padded_segembs.to(args.device),
            mask_src=stories_mask.to(args.device),
            tgt_str=summaries,
        )

        return batch

NLL loss will be higher the smaller the probability of the correct class
What does this all mean? The lower the confidence it has in predicting the correct class, the higher the loss. It will:
1) Penalize correct predictions that it isn't confident about more so than correct predictions it is very confident about.
2) And vice-versa, it will penalize incorrect predictions it is very confident about more so than incorrect predictions it isn't very confident about
Why is this better than accuracy?
Because accuracy simply tells you whether you got it right or wrong (a 1 or a 0), whereast NLL incorporates the confidence as well. That information provides you're model with a much better insight w/r/t to how well it is really doing in a single number (INF to 0), resulting in gradients that the model can actually use!
Rember that a loss function returns a number. That's it!

In [ ]:
class LabelSmoothingCrossEntropy(torch.nn.Module):
    def __init__(self, eps:float, reduction, ignore_index):
        super().__init__()
        self.eps = eps
        self.reduction = reduction
        self.ignore_index = ignore_index

    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        loss = self.reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction, ignore_index=self.ignore_index)
        return self.lin_comb(loss/c, nll, self.eps)

    def reduce_loss(self, loss, reduction='mean'):
        return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

    def lin_comb(self, v1, v2, beta):
        return beta*v1 + (1-beta)*v2

def summaries_loss_fn(inputs, targs):
    loss_fn = (#qua veniva "chiamata" la funzione FlattenedLoss che però è una funzione customizzata che non sono riuscito a trovare da nessuna parte e senza non mi genera comunque errori per ora
        partial(
            LabelSmoothingCrossEntropy, eps=0.1, reduction='mean', ignore_index=tokenizer.pad_token_id
        )
    )
    loss = loss_fn(inputs, targs)
    return loss


def seq2seq_acc(out, targ, pad_idx=-1):
    return (out.argmax(2)==targ).float().mean()


In [ ]:
!pip install transformers==3.5.1 #questo dovrebbe risolvere l'AttributeError sotto secondo stack overflow

In [ ]:
def load_model(pretrained=False, path=None):
    config = BertAbsConfig(max_pos=args.block_size)
    if pretrained:
        if path:
            model = BertAbs.from_pretrained(
                "bertabs-finetuned-cnndm",
                state_dict=torch.load(path, map_location=torch.device(args.device)),
                config=config)
        else:
            model = BertAbs.from_pretrained("remi/bertabs-finetuned-cnndm-extractive-abstractive-summarization", config=config)
    else:
        model = BertAbs(args=config)

    return model.to(args.device)

model = load_model(pretrained=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)
symbols={"BOS": tokenizer.vocab["[unused0]"], "EOS": tokenizer.vocab["[unused1]"]}
data = SummarisationDataset(args.data_path, subset=args.subset)
train_ds, test_ds = train_test_split(data, test_size=1-args.train_pct)
valid_ds, test_ds = train_test_split(test_ds, test_size=0.5)

data = DataBunch.create(
    train_ds,
    valid_ds,
    bs=args.batch_size,
    collate_fn=partial(collate_function, tokenizer=tokenizer, symbols=symbols, block_size=args.block_size, training=True),
)


learn = Learner(
    data,
    model,
    opt_func=partial(Adam, lr=2e-3, betas=(args.adam_b1, args.adam_b2)),
    loss_func=summaries_loss_fn,
    metrics=[seq2seq_acc],
    callback_fns=ShowGraph
)
learn.path = Path('..')
learn = learn.split([model.bert, model.decoder])

learn.freeze_to(-1)
learn.fit_one_cycle(
    1,
    max_lr=5e-3,
    moms=(0.8, 0.7),
    wd=0.1,
    callbacks=[
        SaveModelCallback(learn, every='improvement', monitor='valid_loss', name=f"{args.model_name}_dec")
    ] )

print("mo so cazzi")

In [ ]:
!gsutil cp gs://sfr-books-dataset-chapters-research/all_chapterized_books.zip .
!unzip all_chapterized_books.zip

In [ ]:
!git clone https://github.com/salesforce/booksum/tree/main/scripts/data_collection/novelguide

https://github.com/salesforce/booksum/tree/main/scripts/data_collection per il dataset


In [ ]:
!pip install sentence_transformers==1.2.0
!pip install matplotlib==3.4.2
!pip install numpy==1.19.5
!pip install spacy==3.0.3
!pip install matching==1.4
!pip install nltk==3.6.2
!pip install tqdm==4.49.0
!pip install beautifulsoup4==4.9.3
!pip install Unidecode==1.2.0
!pip install word2number==1.1